In [1]:
import numpy as np
import pandas as pd

In [2]:
import os
os.getcwd()

'/hkfs/work/workspace/scratch/bh6321-E6/infamous-werewolves'

In [43]:
DATA_DIR = "/hkfs/work/workspace/scratch/bh6321-energy_challenge/data/train.csv"
DATA_DIR_DANIEL = "./data/processed/"
TRAIN = "train.csv"
VAL = "valid.csv"

In [75]:
df = pd.read_csv(DATA_DIR_DANIEL+TRAIN)

In [76]:
CITIES = df.city.unique()

In [77]:
df.head()

,load,time,city,hour,day_name,weekday,day,month,year,dayofyear,...,month_cos,dayofyear_sin,dayofyear_cos,latitute,longitude,elevation,is_day,is_dusk,is_dawn,sun_position
0,263.365956,2015-01-01 00:00:00,bs,0,Thursday,3,1,1,2015,1,...,0.866025,0.017213,0.999852,52.266667,10.516667,76.70401,0,0,0,-59.840175
1,259.073621,2015-01-01 01:00:00,bs,1,Thursday,3,1,1,2015,1,...,0.866025,0.017213,0.999852,52.266667,10.516667,76.70401,0,0,0,-55.195090
2,246.612481,2015-01-01 02:00:00,bs,2,Thursday,3,1,1,2015,1,...,0.866025,0.017213,0.999852,52.266667,10.516667,76.70401,0,0,0,-47.964556
3,238.313719,2015-01-01 03:00:00,bs,3,Thursday,3,1,1,2015,1,...,0.866025,0.017213,0.999852,52.266667,10.516667,76.70401,0,0,0,-39.408925
4,237.114390,2015-01-01 04:00:00,bs,4,Thursday,3,1,1,2015,1,...,0.866025,0.017213,0.999852,52.266667,10.516667,76.70401,0,0,0,-30.318257


In [78]:
df.groupby('city')['load'].rolling(20).min()

city        
bs    0               NaN
      1               NaN
      2               NaN
      3               NaN
      4               NaN
                  ...    
wob   210427    51.435961
      210428    51.435961
      210429    51.435961
      210430    51.435961
      210431    51.435961
Name: load, Length: 368256, dtype: float64

In [79]:
city_mapper = {'h': "Hannover", 
               'bs': "Braunschweig", 
               'ol': "Oldenburg", 
               'os': "Osnabrück", 
               'wob': "Wolfsburg", 
               'go': "Göttingen",
               'sz': "Salzgitter", 
               'hi': "Hildesheim", 
               'del': "Delmenhorst", 
               'lg': "Lüneburg", 
               'whv': "Wilhelmshaven", 
               'ce': "Celle", 
               'hm': "Hameln", 
               'el': "Lingen(Ems)"
              }

## 7-day Training

In [108]:
ROLLING_WINDOW = 24

df["load_sq"] = df["load"] ** 2

df_tmp_list = []
for city in CITIES:
    df_tmp = df.loc[df.city == city,:]
    df_tmp.loc[:,"y"] = df_tmp.loc[:,"load"].shift(-7*24)
    df_tmp.dropna(inplace=True)
    df_tmp_list.append(df_tmp)
    
for df_tmp in df_tmp_list:
    df_tmp_len = df_tmp.shape[0]
    df_tmp.loc[:,"load_t-1"] = df_tmp.loc[:,"load"].shift(1)
    df_tmp.loc[:,"load_t-2"] = df_tmp.loc[:,"load"].shift(2)
    df_tmp.loc[:,"load_rmean"] = df_tmp.loc[:,"load"].rolling(ROLLING_WINDOW).mean()
    df_tmp.loc[:,"load_rstd"] = df_tmp.loc[:,"load"].rolling(ROLLING_WINDOW).std()
    df_tmp.loc[:,"load_rmin"] = df_tmp.loc[:,"load"].rolling(ROLLING_WINDOW).min()
    df_tmp.loc[:,"load_rmax"] = df_tmp.loc[:,"load"].rolling(ROLLING_WINDOW).max()
    #df_tmp[["load_t-1", "load_t-2", "load_rmean", "load_rstd", "load_rmin", "load_rmax"]].interpolate(method="polynomial", order=3, inplace=True)
    df_tmp.fillna(method="backfill", inplace=True)
    
    assert df_tmp_len == df_tmp.shape[0]
    assert df_tmp.isnull().values.any() == False
    
df_y = pd.concat(df_tmp_list)
df_y.head()

/software/all/jupyter/tensorflow/2022-01-11/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/software/all/jupyter/tensorflow/2022-01-11/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/software/all/jupyter/tensorflow/2022-01-11/lib/python3.8/site-packages/pandas/core/frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

,load,time,city,hour,day_name,weekday,day,month,year,dayofyear,...,is_dawn,sun_position,load_sq,y,load_t-1,load_t-2,load_rmean,load_rstd,load_rmin,load_rmax
0,263.365956,2015-01-01 00:00:00,bs,0,Thursday,3,1,1,2015,1,...,0,-59.840175,69361.626706,244.871041,263.365956,263.365956,246.43874,11.645996,225.748613,266.161915
1,259.073621,2015-01-01 01:00:00,bs,1,Thursday,3,1,1,2015,1,...,0,-55.195090,67119.140911,240.147987,263.365956,263.365956,246.43874,11.645996,225.748613,266.161915
2,246.612481,2015-01-01 02:00:00,bs,2,Thursday,3,1,1,2015,1,...,0,-47.964556,60817.715963,228.544572,259.073621,263.365956,246.43874,11.645996,225.748613,266.161915
3,238.313719,2015-01-01 03:00:00,bs,3,Thursday,3,1,1,2015,1,...,0,-39.408925,56793.428691,224.794348,246.612481,259.073621,246.43874,11.645996,225.748613,266.161915
4,237.114390,2015-01-01 04:00:00,bs,4,Thursday,3,1,1,2015,1,...,0,-30.318257,56223.233992,233.304757,238.313719,246.612481,246.43874,11.645996,225.748613,266.161915


df_y = pd.get_dummies(df_y, columns=["city",])

In [109]:
df_y.columns

Index(['load', 'time', 'city', 'hour', 'day_name', 'weekday', 'day', 'month',
       'year', 'dayofyear', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos',
       'month_sin', 'month_cos', 'dayofyear_sin', 'dayofyear_cos', 'latitute',
       'longitude', 'elevation', 'is_day', 'is_dusk', 'is_dawn',
       'sun_position', 'load_sq', 'y', 'load_t-1', 'load_t-2', 'load_rmean',
       'load_rstd', 'load_rmin', 'load_rmax'],
      dtype='object')

In [110]:
sun_features = ["latitute", "longitude", "elevation", "is_day", "is_dusk", "is_dawn", "sun_position"]

In [111]:
FEATURES = list(df_y.columns)
FEATURES.remove("y")
FEATURES.remove("day_name")
FEATURES.remove("city")
FEATURES.remove("time")
for f in sun_features:
    FEATURES.remove(f)

In [112]:
df_tmp_list[0].shape

(26136, 33)

In [113]:
from sklearn.ensemble import RandomForestRegressor

In [114]:
import xgboost as xgb
from xgboost import XGBRegressor

In [115]:
len(df_tmp_list)

14

In [116]:
df_tmp_list[0]

,load,time,city,hour,day_name,weekday,day,month,year,dayofyear,...,is_dawn,sun_position,load_sq,y,load_t-1,load_t-2,load_rmean,load_rstd,load_rmin,load_rmax
0,263.365956,2015-01-01 00:00:00,bs,0,Thursday,3,1,1,2015,1,...,0,-59.840175,69361.626706,244.871041,263.365956,263.365956,246.438740,11.645996,225.748613,266.161915
1,259.073621,2015-01-01 01:00:00,bs,1,Thursday,3,1,1,2015,1,...,0,-55.195090,67119.140911,240.147987,263.365956,263.365956,246.438740,11.645996,225.748613,266.161915
2,246.612481,2015-01-01 02:00:00,bs,2,Thursday,3,1,1,2015,1,...,0,-47.964556,60817.715963,228.544572,259.073621,263.365956,246.438740,11.645996,225.748613,266.161915
3,238.313719,2015-01-01 03:00:00,bs,3,Thursday,3,1,1,2015,1,...,0,-39.408925,56793.428691,224.794348,246.612481,259.073621,246.438740,11.645996,225.748613,266.161915
4,237.114390,2015-01-01 04:00:00,bs,4,Thursday,3,1,1,2015,1,...,0,-30.318257,56223.233992,233.304757,238.313719,246.612481,246.438740,11.645996,225.748613,266.161915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26131,218.972219,2017-12-24 19:00:00,bs,19,Sunday,6,24,12,2017,358,...,0,-33.808671,47948.832654,208.679526,228.217820,232.539860,213.465857,14.068149,186.241306,232.539860
26132,213.599374,2017-12-24 20:00:00,bs,20,Sunday,6,24,12,2017,358,...,0,-42.785189,45624.692443,202.987355,218.972219,228.217820,213.262411,14.027700,186.241306,232.539860
26133,220.502013,2017-12-24 21:00:00,bs,21,Sunday,6,24,12,2017,358,...,0,-51.006193,48621.137791,210.042231,213.599374,218.972219,213.194492,13.986775,186.241306,232.539860
26134,229.740188,2017-12-24 22:00:00,bs,22,Sunday,6,24,12,2017,358,...,0,-57.526141,52780.553932,218.871965,220.502013,213.599374,213.279119,14.084219,186.241306,232.539860


In [117]:
regressors = {}

for df_tmp in df_tmp_list:
    
    city = df_tmp.city.unique()[0]
    
    print("City: ", city)
    
    X = df_tmp[FEATURES]
    y = df_tmp["y"]
    
    #regr = RandomForestRegressor(n_estimators=2000,
    #                             max_depth=20, 
    #                             criterion="squared_error",
    #                             max_features="sqrt",
    #                             random_state=7531,
    #                             oob_score=False,
    #                             n_jobs=-1)
    
    regr = XGBRegressor(objective='reg:squarederror', 
                        n_jobs=-1)
    
    #params = {'colsample_bytree': 0.049, 
    #         'learning_rate': 0.0113, 
    #         'max_depth': 9, 
    #         'n_estimators': 7000, 
    #         'subsample': 0.8}

    #regr.set_params(**params)
    
    regr.fit(X,y)
    
    regressors[city] = regr
print("DONE")

City:  bs
City:  h
City:  hm
City:  ol
City:  del
City:  el
City:  os
City:  wob
City:  hi
City:  whv
City:  lg
City:  ce
City:  go
City:  sz
DONE


## Validation

In [127]:
df_val = pd.read_csv(DATA_DIR_DANIEL+VAL)
df_val["load_sq"] = df_val["load"] ** 2

df_tmp_list_val = []
for city in CITIES:
    df_tmp = df_val.loc[df_val.city == city,:]
    df_tmp.loc[:,"y"] = df_tmp.loc[:,"load"].shift(-7*24)
    df_tmp.dropna(inplace=True)
    df_tmp_list_val.append(df_tmp)
    
for df_tmp in df_tmp_list_val:
    df_tmp_len = df_tmp.shape[0]
    df_tmp.loc[:,"load_t-1"] = df_tmp.loc[:,"load"].shift(1)
    df_tmp.loc[:,"load_t-2"] = df_tmp.loc[:,"load"].shift(2)
    df_tmp.loc[:,"load_rmean"] = df_tmp.loc[:,"load"].rolling(ROLLING_WINDOW).mean()
    df_tmp.loc[:,"load_rstd"] = df_tmp.loc[:,"load"].rolling(ROLLING_WINDOW).std()
    df_tmp.loc[:,"load_rmin"] = df_tmp.loc[:,"load"].rolling(ROLLING_WINDOW).min()
    df_tmp.loc[:,"load_rmax"] = df_tmp.loc[:,"load"].rolling(ROLLING_WINDOW).max()
    #df_tmp[["load_t-1", "load_t-2", "load_rmean", "load_rstd", "load_rmin", "load_rmax"]].interpolate(method="polynomial", order=3, inplace=True)
    df_tmp.fillna(method="backfill", inplace=True)
    
    assert df_tmp_len == df_tmp.shape[0]
    assert df_tmp.isnull().values.any() == False
    
df_val = pd.concat(df_tmp_list_val)
df_val.head()

/software/all/jupyter/tensorflow/2022-01-11/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/software/all/jupyter/tensorflow/2022-01-11/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/software/all/jupyter/tensorflow/2022-01-11/lib/python3.8/site-packages/pandas/core/frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

,load,time,city,hour,day_name,weekday,day,month,year,dayofyear,...,is_dawn,sun_position,load_sq,y,load_t-1,load_t-2,load_rmean,load_rstd,load_rmin,load_rmax
0,208.066865,2018-01-01 00:00:00,bs,0,Monday,0,1,1,2018,1,...,0,-59.825798,43291.820516,212.218103,208.066865,208.066865,202.704386,14.454036,181.941545,227.556889
1,202.341276,2018-01-01 01:00:00,bs,1,Monday,0,1,1,2018,1,...,0,-55.190329,40941.992163,207.253698,208.066865,208.066865,202.704386,14.454036,181.941545,227.556889
2,191.499046,2018-01-01 02:00:00,bs,2,Monday,0,1,1,2018,1,...,0,-47.965888,36671.884477,197.562526,202.341276,208.066865,202.704386,14.454036,181.941545,227.556889
3,183.315389,2018-01-01 03:00:00,bs,3,Monday,0,1,1,2018,1,...,0,-39.413265,33604.531919,195.427498,191.499046,202.341276,202.704386,14.454036,181.941545,227.556889
4,181.941545,2018-01-01 04:00:00,bs,4,Monday,0,1,1,2018,1,...,0,-30.323468,33102.725755,206.770996,183.315389,191.499046,202.704386,14.454036,181.941545,227.556889


In [128]:
from sklearn.metrics import mean_absolute_error as mae

In [129]:
def mae2(y_pred, y_true):
    return np.mean(np.abs(y_pred - y_true))

In [133]:
val_scores = []
for df_tmp in df_tmp_list_val:
    city = df_tmp.city.unique()[0]
    pred = regressors[city].predict(df_tmp[FEATURES])
    #pred = df_tmp["load"]
    score = mae(pred, df_tmp["y"])
    val_scores.append(score)

In [134]:
val_scores

[2.057954763188023,
 6.890915254704192,
 1.084267879310362,
 2.192322946673322,
 1.0488069196080103,
 0.5792244899922098,
 0.9626223259454636,
 0.6985026612090164,
 0.5163852493933946,
 0.791232276621311,
 2.395972248717909,
 0.46914912866025565,
 2.998122508656814,
 3.172703355459504]

In [135]:
np.mean(val_scores)

1.8470130005814132

In [26]:
np.mean(val_scores)

2.647748514198228

In [124]:
regressors = {}

for df_train, df_val in zip(df_tmp_list, df_tmp_list_val):
    
    assert df_train.city.unique()[0] == df_val.city.unique()[0]
    
    city = df_train.city.unique()[0]
    
    print("City: ", city)
    
    df_tmp = pd.concat([df_train, df_val])
    
    X = df_tmp[FEATURES]
    y = df_tmp["y"]
    
    regr = RandomForestRegressor(n_estimators=2000,
                                 max_depth=20, 
                                 criterion="squared_error",
                                 max_features="sqrt",
                                 random_state=7531,
                                 oob_score=False,
                                 n_jobs=-1)
    
    regr.fit(X,y)
    
    regressors[city] = regr
print("DONE")

City:  bs
City:  h
City:  hm
City:  ol
City:  del
City:  el
City:  os
City:  wob
City:  hi
City:  whv
City:  lg
City:  ce
City:  go
City:  sz
DONE


In [125]:
import pickle

In [126]:
with open('model_dict.pickle', 'wb') as handle:
    pickle.dump(regressors, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [136]:
with open('model_dict.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [137]:
d1 = dict(list(b.items())[len(b)//2:])
d2 = dict(list(b.items())[:len(b)//2])

In [138]:
with open('model_dict_1.pickle', 'wb') as handle:
    pickle.dump(d1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('model_dict_2.pickle', 'wb') as handle:
    pickle.dump(d2, handle, protocol=pickle.HIGHEST_PROTOCOL)